In [34]:
# Librerías

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import datasets
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import pickle
import joblib

from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

from sklearn import preprocessing



In [37]:
# CSV EDA
df=pd.read_csv('./data/competi_kaggle.csv')
df
 


,index_id,depth,table,x,y,z,color,clarity,city,cut,carat,price
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,62.4,58.0,6.83,6.79,4.25,J,VS2,Kimberly,Premium,1.21,4268
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,63.0,57.0,4.35,4.38,2.75,H,VS2,Antwerp,Very Good,0.32,505
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,65.5,55.0,5.62,5.53,3.65,G,VS1,London,Fair,0.71,2686
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,63.8,56.0,4.68,4.72,3.00,D,SI1,New York City,Good,0.41,738
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,60.5,59.0,6.55,6.51,3.95,G,SI1,Dubai,Ideal,1.02,4882
...,...,...,...,...,...,...,...,...,...,...,...,...
40450,f0bc79169405ebeb24e308055156b946ffd819db9b4f75...,62.7,57.0,7.10,7.04,4.43,G,VS1,New York City,Ideal,1.34,10070
40451,339916a23bf22b052b54cb2a9b36ee8418c1c68b46acad...,57.1,60.0,8.31,8.25,4.73,F,SI2,Zurich,Good,2.02,12615
40452,46957922b99954654c1deb8d854c3f069bf118b2ce9415...,62.7,56.0,6.37,6.42,4.01,H,SI1,Tel Aviv,Ideal,1.01,5457
40453,9d733392d362d5c6f1d9b9659b601c7d4b5a1c1c8df579...,61.9,54.3,4.45,4.47,2.76,J,VS1,Paris,Ideal,0.33,456


# Preprocesing

In [38]:
df.describe()

,depth,table,x,y,z,carat,price
count,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000
mean,61.752841,57.446133,5.729392,5.732819,3.537154,0.797706,3928.444469
std,1.431725,2.233535,1.124453,1.146650,0.697062,0.475544,3992.416147
min,43.000000,43.000000,0.000000,0.000000,0.000000,0.200000,326.000000
25%,61.000000,56.000000,4.710000,4.720000,2.910000,0.400000,945.000000
50%,61.800000,57.000000,5.690000,5.710000,3.520000,0.700000,2397.000000
75%,62.500000,59.000000,6.540000,6.540000,4.035000,1.040000,5331.000000
max,79.000000,95.000000,10.230000,58.900000,8.060000,4.500000,18823.000000


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40455 entries, 0 to 40454
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   index_id  40455 non-null  object 
 1   depth     40455 non-null  float64
 2   table     40455 non-null  float64
 3   x         40455 non-null  float64
 4   y         40455 non-null  float64
 5   z         40455 non-null  float64
 6   color     40455 non-null  object 
 7   clarity   40455 non-null  object 
 8   city      40455 non-null  object 
 9   cut       40455 non-null  object 
 10  carat     40455 non-null  float64
 11  price     40455 non-null  int64  
dtypes: float64(6), int64(1), object(5)
memory usage: 3.7+ MB


In [40]:
df.corr()

,depth,table,x,y,z,carat,price
depth,1.000000,-0.293114,-0.026348,-0.030966,0.094655,0.026528,-0.014864
table,-0.293114,1.000000,0.196059,0.184673,0.155189,0.183392,0.130111
x,-0.026348,0.196059,1.000000,0.973712,0.984876,0.975688,0.885848
y,-0.030966,0.184673,0.973712,1.000000,0.964828,0.951667,0.866163
z,0.094655,0.155189,0.984876,0.964828,1.000000,0.967570,0.874500
carat,0.026528,0.183392,0.975688,0.951667,0.967570,1.000000,0.921935
price,-0.014864,0.130111,0.885848,0.866163,0.874500,0.921935,1.000000


# Feature Engennering

In [41]:
X = df[['depth', 'table','color', 'clarity','x', 'y', 'z',  'cut', 'carat']]
X

,depth,table,color,clarity,x,y,z,cut,carat
0,62.4,58.0,J,VS2,6.83,6.79,4.25,Premium,1.21
1,63.0,57.0,H,VS2,4.35,4.38,2.75,Very Good,0.32
2,65.5,55.0,G,VS1,5.62,5.53,3.65,Fair,0.71
3,63.8,56.0,D,SI1,4.68,4.72,3.00,Good,0.41
4,60.5,59.0,G,SI1,6.55,6.51,3.95,Ideal,1.02
...,...,...,...,...,...,...,...,...,...
40450,62.7,57.0,G,VS1,7.10,7.04,4.43,Ideal,1.34
40451,57.1,60.0,F,SI2,8.31,8.25,4.73,Good,2.02
40452,62.7,56.0,H,SI1,6.37,6.42,4.01,Ideal,1.01
40453,61.9,54.3,J,VS1,4.45,4.47,2.76,Ideal,0.33


In [42]:
y = df[['price']]
y


,price
0,4268
1,505
2,2686
3,738
4,4882
...,...
40450,10070
40451,12615
40452,5457
40453,456


## Encoding

In [43]:
# One hot encoding
cat_cols = [ 'color','clarity', 'cut' ]
df_one_hot = pd.get_dummies(X, columns=cat_cols, drop_first=True)
df_one_hot

,depth,table,x,y,z,carat,color_E,color_F,color_G,color_H,...,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2,cut_Good,cut_Ideal,cut_Premium,cut_Very Good
0,62.4,58.0,6.83,6.79,4.25,1.21,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,63.0,57.0,4.35,4.38,2.75,0.32,0,0,0,1,...,0,0,0,1,0,0,0,0,0,1
2,65.5,55.0,5.62,5.53,3.65,0.71,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
3,63.8,56.0,4.68,4.72,3.00,0.41,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
4,60.5,59.0,6.55,6.51,3.95,1.02,0,0,1,0,...,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,62.7,57.0,7.10,7.04,4.43,1.34,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0
40451,57.1,60.0,8.31,8.25,4.73,2.02,0,1,0,0,...,0,1,0,0,0,0,1,0,0,0
40452,62.7,56.0,6.37,6.42,4.01,1.01,0,0,0,1,...,1,0,0,0,0,0,0,1,0,0
40453,61.9,54.3,4.45,4.47,2.76,0.33,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0


## Scaling

In [44]:
# Scaling Standard
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_one_hot)
#scaled_data = scaler.fit_transform(X)
scaled_data

#MinMax Scaler
#scaler = MinMaxScaler()
#scaled_data = scaler.fit_transform(df_one_hot[['depth', 'table','carat']])
#scaled_data

# Robust Scaling
#scaler = RobustScaler()
#scaled_data = scaler.fit_transform(df_one_hot[['depth', 'table','carat']])
#scaled_data

array([[ 0.45201864,  0.24798091,  0.97880679, ..., -0.81809533,
         1.7155123 , -0.53853424],
       [ 0.8710986 , -0.19974534, -1.22673789, ..., -0.81809533,
        -0.58291625,  1.85689213],
       [ 2.61726508, -1.09519783, -0.09728557, ..., -0.81809533,
        -0.58291625, -0.53853424],
       ...,
       [ 0.66155862, -0.64747158,  0.56971383, ...,  1.22235144,
        -0.58291625, -0.53853424],
       [ 0.10278535, -1.4086062 , -1.13780463, ...,  1.22235144,
        -0.58291625, -0.53853424],
       [ 0.172632  ,  0.24798091,  0.97880679, ...,  1.22235144,
        -0.58291625, -0.53853424]])

In [45]:
df_X=pd.DataFrame(scaled_data)
df_X

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,0.452019,0.247981,0.978807,0.921985,1.022657,0.867006,-0.470211,-0.464401,-0.514022,-0.428551,...,-0.563467,-0.454616,-0.419992,1.836970,-0.271326,-0.32193,-0.315531,-0.818095,1.715512,-0.538534
1,0.871099,-0.199745,-1.226738,-1.179816,-1.129259,-1.004557,-0.470211,-0.464401,-0.514022,2.333443,...,-0.563467,-0.454616,-0.419992,1.836970,-0.271326,-0.32193,-0.315531,-0.818095,-0.582916,1.856892
2,2.617265,-1.095198,-0.097286,-0.176882,0.161891,-0.184434,-0.470211,-0.464401,1.945442,-0.428551,...,-0.563467,-0.454616,2.380995,-0.544375,-0.271326,-0.32193,-0.315531,-0.818095,-0.582916,-0.538534
3,1.429872,-0.647472,-0.933258,-0.883296,-0.770607,-0.815298,-0.470211,-0.464401,-0.514022,-0.428551,...,1.774727,-0.454616,-0.419992,-0.544375,-0.271326,-0.32193,3.169263,-0.818095,-0.582916,-0.538534
4,-0.875068,0.695707,0.729794,0.677793,0.592274,0.467458,-0.470211,-0.464401,1.945442,-0.428551,...,1.774727,-0.454616,-0.419992,-0.544375,-0.271326,-0.32193,-0.315531,1.222351,-0.582916,-0.538534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,0.661559,-0.199745,1.218927,1.140014,1.280887,1.140380,-0.470211,-0.464401,1.945442,-0.428551,...,-0.563467,-0.454616,2.380995,-0.544375,-0.271326,-0.32193,-0.315531,1.222351,-0.582916,-0.538534
40451,-3.249854,1.143433,2.295019,2.195276,1.711271,2.570338,-0.470211,2.153313,-0.514022,-0.428551,...,-0.563467,2.199660,-0.419992,-0.544375,-0.271326,-0.32193,3.169263,-0.818095,-0.582916,-0.538534
40452,0.661559,-0.647472,0.569714,0.599302,0.678351,0.446430,-0.470211,-0.464401,-0.514022,2.333443,...,1.774727,-0.454616,-0.419992,-0.544375,-0.271326,-0.32193,-0.315531,1.222351,-0.582916,-0.538534
40453,0.102785,-1.408606,-1.137805,-1.101325,-1.114913,-0.983529,-0.470211,-0.464401,-0.514022,-0.428551,...,-0.563467,-0.454616,2.380995,-0.544375,-0.271326,-0.32193,-0.315531,1.222351,-0.582916,-0.538534


In [46]:
%%time

# Model definition

#model = LinearRegression()
model = (RandomForestRegressor(bootstrap=True, random_state = 42))
#model = linear_model.Lasso()
#model = ElasticNet()
#model = Ridge()
#model = SVR()
#model = SGDRegressor()
#model = DecisionTreeRegressor()


hyperparameters = model.get_params()



print(type(model), '\n')
print('Model hyperparameters:', hyperparameters, '\n')

<class 'sklearn.ensemble._forest.RandomForestRegressor'> 

Model hyperparameters: {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False} 

CPU times: total: 46.9 ms
Wall time: 45.9 ms


In [47]:
%%time

# Model training

#model.model_transform(df_X, y)

model.fit(df_X, y)
#y_pred = model.predict(y)

print('Model:', model, '\n')
print('Model hyperparameters:', hyperparameters, '\n')
#print('Model coefficients:', model.coef_, '\n')
#print('Ground truth target:', y_test, '\n')
#print('Ground truth target:', y_pred, '\n')

<timed exec>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Model: RandomForestRegressor(random_state=42) 

Model hyperparameters: {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False} 

CPU times: total: 30.3 s
Wall time: 31.4 s


## Metrics Analyses

In [48]:
# Train and test split

X_train, X_test, y_train, y_test = train_test_split(df_X, y, test_size=0.2, random_state=42)
print(f"X_train: {X_train.shape}, X_test: {X_test.shape}, y_train: {y_train.shape}, y_test: {y_test.shape}")

X_train: (32364, 23), X_test: (8091, 23), y_train: (32364, 1), y_test: (8091, 1)


In [49]:
y_pred = model.predict(X_test)

In [50]:
# MSE calculation

mean_squared_error(y_test, y_pred)

53263.83586094317

In [51]:
# RMSE calculation 

rmse = mean_squared_error(y_test, y_pred, squared=False)
rmse

230.7895921850532

In [52]:
#Nota: 230

## Visualitation

## Kaggle

In [54]:
# CSV diamonds_test Kaggle
df_K=pd.read_csv('./data/diamonds_test.csv')
df_K

,id,carat,cut,color,clarity,depth,table,x,y,z,city
0,0,0.79,Very Good,F,SI1,62.7,60.0,5.82,5.89,3.67,Amsterdam
1,1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,4.18,Surat
2,2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,4.57,Kimberly
3,3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,3.90,Kimberly
4,4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,3.19,Amsterdam
...,...,...,...,...,...,...,...,...,...,...,...
13480,13480,0.57,Ideal,E,SI1,61.9,56.0,5.35,5.32,3.30,Amsterdam
13481,13481,0.71,Ideal,I,VS2,62.2,55.0,5.71,5.73,3.56,New York City
13482,13482,0.70,Ideal,F,VS1,61.6,55.0,5.75,5.71,3.53,Tel Aviv
13483,13483,0.70,Very Good,F,SI2,58.8,57.0,5.85,5.89,3.45,Surat


In [ ]:
X_K = df_K[['depth', 'table','color', 'clarity','x','y','z','cut', 'carat']]
X_K


In [ ]:
# One hot encoding
cat_cols = [ 'color','clarity', 'cut' ]
df_one_hot_K = pd.get_dummies(X_K, columns=cat_cols, drop_first=True)
df_one_hot_K

In [ ]:
# Scaling columnas numéricas
scaler = StandardScaler()
scaled_data_K = scaler.fit_transform(df_one_hot_K)
scaled_data_K

In [ ]:
df_X_K=pd.DataFrame(scaled_data_K)
df_X_K

In [ ]:
# Model predictions

#predictions = model.predict(df_Diamond_test)
y_pred = model.predict(df_X_K)

print(type(y_pred))

In [ ]:
y_pred


In [ ]:
df_predictions = pd.DataFrame(y_pred, columns=['price']).reset_index()
df_predictions

In [ ]:
df_predictions['id']=df_predictions.index
df_predictions

In [ ]:
df_predictions_d=df_predictions.drop('index', axis=1)
df_predictions_d

In [ ]:
df_predictions_d.to_csv('train.csv', index=False)